<a href="https://colab.research.google.com/github/aaubs/ds22/blob/master/notebooks/m3_ws_intro_tstf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# before starting, we define some constants
figsize = (9, 6)
lowest_q, low_q, high_q, highest_q = 0.01, 0.1, 0.9, 0.99
label_q_outer = f"{int(lowest_q * 100)}-{int(highest_q * 100)}th percentiles"
label_q_inner = f"{int(low_q * 100)}-{int(high_q * 100)}th percentiles"

# Introduction to Transformer-based Timeseries Forecast

# Temporal Fusion Transformers

## Introduction

xxx
xxx
xxx

## Timesieries implementation (DARTS)

darts is a Python library for easy manipulation and forecasting of time series. It contains a variety of models, from classics such as ARIMA to deep neural networks. The models can all be used in the same way, using fit() and predict() functions, similar to scikit-learn. The library also makes it easy to backtest models, combine the predictions of several models, and take external data into account. Darts supports both univariate and multivariate time series and models. The ML-based models can be trained on potentially large datasets containing multiple time series, and some of the models offer a rich support for probabilistic forecasting.

In [ ]:
#!pip install darts

In [ ]:
from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, IceCreamHeaterDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression

import warnings
warnings.filterwarnings("ignore")

Darts’ TFTModel incorporates the following main components from the original Temporal Fusion Transformer (TFT) architecture:

*gating mechanisms: skip over unused components of the model architecture
* variable selection networks: select relevant input variables at each time step.
* temporal processing of past and future input with LSTMs (long short-term memory)
* multi-head attention: captures long-term temporal dependencies
* prediction intervals: per default, produces quantile forecasts instead of deterministic values

### Training

TFTModel can be trained with past and future covariates. It is trained sequentially on fixed-size chunks consisting of an encoder and a decoder part:

* encoder: past input with input_chunk_length
  * past target: mandatory
  * past covariates: optional
* decoder: future known input with output_chunk_length
  * future covariates: mandatory (if none are available, consider TFTModel’s optional arguments add_encoders or add_relative_index from here)

In each iteration, the model produces a quantile prediction of shape (output_chunk_length, n_quantiles) on the decoder part.

### Forecast

Per default, TFTModel produces probabilistic quantile forecasts using QuantileRegression. This gives the range of likely target values at each prediction step. Most deep learning models in Darts’ - including TFTModel - support QuantileRegression and 16 other likelihoods to produce probabilistic forecasts by setting likelihood=MyLikelihood() at model creation.

### Toy example (Air Passangers)

Adopted from the [DARTS pakage tutorial](https://unit8co.github.io/darts/examples/13-TFT-examples.html)



This data set that is highly dependent on covariates. Knowing the month tells us a lot about the seasonal component, whereas the year determines the effect of the trend component.

Additionally, let’s convert the time index to integer values and use them as covariates as well.

All of the three covariates are known in the future, and can be used as future_covariates with the TFTModel.

In [ ]:
# Read data
series = AirPassengersDataset().load()

In [ ]:
series.head()

In [ ]:
# we convert monthly number of passengers to average daily number of passengers per month
series = series / TimeSeries.from_series(series.time_index.days_in_month)
series = series.astype(np.float32)

In [ ]:
# Create training and validation sets:
training_cutoff = pd.Timestamp("19571201")
train, val = series.split_after(training_cutoff)

In [ ]:
# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

In [ ]:
# create year, month and integer index covariate series
covariates = datetime_attribute_timeseries(series, attribute="year", one_hot=False)

In [ ]:
covariates = covariates.stack(datetime_attribute_timeseries(series, attribute="month", one_hot=False))

In [ ]:
covariates = covariates.stack(
    TimeSeries.from_times_and_values(
        times=series.time_index,
        values=np.arange(len(series)),
        columns=["linear_increase"],
    )
)

covariates = covariates.astype(np.float32)

In [ ]:
cov_train, cov_val = covariates.split_after(training_cutoff)

In [ ]:
# transform covariates (note: we fit the transformer on train split and can then transform the entire covariates series)
scaler_covs = Scaler()
scaler_covs.fit(cov_train)
covariates_transformed = scaler_covs.transform(covariates)

The TFTModel can only be used if some future input is given. Optional parameters add_encoders and add_relative_index can be useful, especially if we don’t have any future input available. They generate endoded temporal data is used as future covariates.

Since we already have future covariates defined in our example they are commented out.

In [ ]:
num_samples = 200
input_chunk_length = 24
forecast_horizon = 12

In [ ]:
my_model = TFTModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=forecast_horizon,
    hidden_size=64,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=16,
    n_epochs=300,
    add_relative_index=False,
    add_encoders=None,
    likelihood=QuantileRegression(
        # quantiles= [ 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
    ),  # QuantileRegression is set per default
    # loss_fn=MSELoss(),
    random_state=42,
)

In what follows, we can just provide the whole covariates series as future_covariates argument to the model; the model will slice these covariates and use only what it needs in order to train on forecasting the target train_transformed:

In [ ]:
my_model.fit(train_transformed, future_covariates=covariates_transformed, verbose=True)

We perform a one-shot prediction of 24 months using the “current” model - i.e., the model at the end of the training procedure:

In [ ]:
def eval_model(model, n, actual_series, val_series):
    pred_series = model.predict(n=n, num_samples=num_samples)

    # plot actual series
    plt.figure(figsize=figsize)
    actual_series[: pred_series.end_time()].plot(label="actual")

    # plot prediction with quantile ranges
    pred_series.plot(
        low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer
    )
    pred_series.plot(low_quantile=low_q, high_quantile=high_q, label=label_q_inner)

    plt.title("MAPE: {:.2f}%".format(mape(val_series, pred_series)))
    plt.legend()

In [ ]:
eval_model(my_model, 24, series_transformed, val_transformed)

Let’s backtest our TFTModel model, to see how it performs with a forecast horizon of 12 months over the last 3 years:

In [ ]:
backtest_series = my_model.historical_forecasts(
    series_transformed,
    future_covariates=covariates_transformed,
    start=train.end_time() + train.freq,
    num_samples=num_samples,
    forecast_horizon=forecast_horizon,
    stride=forecast_horizon,
    last_points_only=False,
    retrain=False,
    verbose=True,
)

In [ ]:
def eval_backtest(backtest_series, actual_series, horizon, start, transformer):
    plt.figure(figsize=figsize)
    actual_series.plot(label="actual")
    backtest_series.plot(
        low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer
    )
    backtest_series.plot(low_quantile=low_q, high_quantile=high_q, label=label_q_inner)
    plt.legend()
    plt.title(f"Backtest, starting {start}, {horizon}-months horizon")
    print(
        "MAPE: {:.2f}%".format(
            mape(
                transformer.inverse_transform(actual_series),
                transformer.inverse_transform(backtest_series),
            )
        )
    )

In [ ]:
eval_backtest(
    backtest_series=concatenate(backtest_series),
    actual_series=series_transformed,
    horizon=forecast_horizon,
    start=training_cutoff,
    transformer=transformer,
)